<a href="https://colab.research.google.com/github/Shinde-iru1920/images/blob/main/Real_ESRGAN_Image_and_Video_Upscaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
# @title
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop
!pip install tqdm

from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 720):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 39.62 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN/Real-ESRGAN
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/dist-packages/setuptools/command/develop.py:41: EasyInstallDep

In [20]:
import os
import subprocess

# Define the path to the file
# Use pip show to find the installation location of basicsr
try:
    # Run pip show basicsr and capture the output
    result = subprocess.run(['pip', 'show', 'basicsr'], capture_output=True, text=True, check=True)
    # Find the 'Location:' line in the output
    for line in result.stdout.splitlines():
        if line.startswith('Location:'):
            package_location = line.split(': ')[1].strip()
            break
    else:
        raise Exception("Could not find installation location for basicsr")

    # Construct the file path using the found location
    file_path = os.path.join(package_location, 'basicsr', 'data', 'degradations.py')
    print(f"Constructed file path: {file_path}")

except Exception as e:
    print(f"Error finding basicsr installation location: {e}")
    # Exit or handle the error appropriately
    exit()


# Check if the file exists before attempting to open it
if not os.path.exists(file_path):
    print(f"Error: File not found at the constructed path: {file_path}")
    # Exit or handle the error appropriately
    exit()

# Read the content of the file
with open(file_path, 'r') as file:
    file_data = file.readlines()

# Modify the specific line
modified = False
for i, line in enumerate(file_data):
    if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
        file_data[i] = 'from torchvision.transforms.functional import rgb_to_grayscale\n'
        modified = True
        break

if modified:
    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.writelines(file_data)
    print("Modification applied successfully.")
else:
    print("Target line not found in the file. No modification applied.")

Constructed file path: /usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py
Target line not found in the file. No modification applied.


In [22]:
!pip install tqdm

In [ ]:
import os
import subprocess
from tqdm import tqdm  # progress bar
import time

# User settings
input_folder = "/content/drive/MyDrive/Manga/demon slayer manga vol.17 cha-143 - 151 Folder" # @param {type:"string"}
output_folder = "/content/drive/MyDrive/Manga/demon slayer manga vol.17 cha-143 - 151 Folder - upscaled" # @param {type:"string"}
model = "RealESRGAN_x4plus_anime_6B" # @param ["RealESRGAN_x4plus", "RealESRGAN_x2plus", "RealESRNet_x4plus", "official ESRGAN_x4", "realesr-general-x4v3", "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]
scale = 4 # @param {type:"slider", min:1, max:4, step:1}
face_enhance = False # @param {type:"boolean"}
code = "inference_realesrgan.py"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Collect image files
image_extensions = (".png", ".jpg", ".jpeg", ".webp")
input_files = [f for f in os.listdir(input_folder) if f.lower().endswith(image_extensions)]

print(f"Found {len(input_files)} image(s). Starting batch processing...\n")

# List to keep track of any failed files
failed_files = []

# Processing with progress bar
for filename in tqdm(input_files, desc="Upscaling", unit="image"):
    input_path = os.path.join(input_folder, filename)

    # Construct command
    if face_enhance:
        command = f"python {code} -i '{input_path}' -n {model} -o '{output_folder}' -s {scale} --face_enhance"
    else:
        command = f"python {code} -i '{input_path}' -n {model} -o '{output_folder}' -s {scale}"

    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"\n❌ Failed to process: {filename} | Error: {e}")
        failed_files.append(filename)

    time.sleep(0.2)  # Optional cooldown

# Summary
print(f"\n✅ Done! Processed {len(input_files) - len(failed_files)} of {len(input_files)} images.")
if failed_files:
    print(f"\n❌ {len(failed_files)} image(s) failed:")
    for f in failed_files:
        print(" -", f)


Found 174 image(s). Starting batch processing...



Upscaling:   2%|▏         | 3/174 [00:40<37:24, 13.12s/image]